In [3]:
from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

np.random.seed(42)

In [4]:
(x_train, _), (x_test, _) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [5]:
image_size = x_train.shape[1]

In [6]:
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [7]:
noise = np.random.normal(loc=0.5, scale=0.5, size=x_train.shape)
x_train_noisy = x_train + noise

noise = np.random.normal(loc=0.5, scale=0.5, size=x_test.shape)
x_test_noisy = x_test + noise

In [8]:
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [9]:
input_shape = (image_size, image_size, 1)
batch_size = 32
kernel_size = 3
latent_dim = 16

In [10]:
layer_filters = [32, 64]

In [11]:
inputs = Input(shape=input_shape, name='encoder_input')
x = inputs

In [12]:
for filters in layer_filters:
  x = Conv2D(filters=filters, kernel_size=kernel_size, strides=2, activation='relu', padding='same')(x)

In [13]:
shape = K.int_shape(x)

In [14]:
x = Flatten()(x)
latent = Dense(latent_dim, name='latent_vector')(x)

In [15]:
encoder = Model(inputs, latent, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 latent_vector (Dense)       (None, 16)                50192     
                                                                 
Total params: 69,008
Trainable params: 69,008
Non-trainable params: 0
_________________________________________________________________


In [16]:
latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(shape[1] * shape[2] * shape[3])(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)

In [18]:
for filters in layer_filters[::-1]:
  x = Conv2DTranspose(filters=filters, kernel_size=kernel_size, strides=2, activation='relu', padding='same')(x)

In [20]:
outputs = Conv2DTranspose(filters=1, kernel_size=kernel_size, padding='same', activation='sigmoid', name='decoder_output')(x)

In [21]:
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 16)]              0         
                                                                 
 dense (Dense)               (None, 3136)              53312     
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 64)       36928     
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 28, 28, 32)       18464     
 ranspose)                                                       
                                                                 
 decoder_output (Conv2DTrans  (None, 28, 28, 1)        289 

In [22]:
autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder (Functional)        (None, 16)                69008     
                                                                 
 decoder (Functional)        (None, 28, 28, 1)         108993    
                                                                 
Total params: 178,001
Trainable params: 178,001
Non-trainable params: 0
_________________________________________________________________


In [23]:
autoencoder.compile(loss='mse', optimizer='adam')

In [24]:
autoencoder.fit(x_train_noisy, x_train, validation_data=(x_test_noisy, x_test), epochs=10, batch_size=batch_size)

Epoch 1/10
1875/1875 [==============================] - 124s 65ms/step - loss: 0.0357 - val_loss: 0.0206
Epoch 2/10
1875/1875 [==============================] - 119s 64ms/step - loss: 0.0194 - val_loss: 0.0182
Epoch 3/10
1875/1875 [==============================] - 123s 65ms/step - loss: 0.0179 - val_loss: 0.0175
Epoch 4/10
1875/1875 [==============================] - 125s 67ms/step - loss: 0.0171 - val_loss: 0.0170
Epoch 5/10
1875/1875 [==============================] - 128s 68ms/step - loss: 0.0166 - val_loss: 0.0165
Epoch 6/10
1875/1875 [==============================] - 129s 69ms/step - loss: 0.0162 - val_loss: 0.0165
Epoch 7/10
1875/1875 [==============================] - 129s 69ms/step - loss: 0.0159 - val_loss: 0.0161
Epoch 8/10
1875/1875 [==============================] - 129s 69ms/step - loss: 0.0157 - val_loss: 0.0160
Epoch 9/10
1875/1875 [==============================] - 131s 70ms/step - loss: 0.0155 - val_loss: 0.0157
Epoch 10/10
1875/1875 [==============================] 

In [25]:
x_decoded = autoencoder.predict(x_test_noisy)

313/313 [==============================] - 7s 21ms/step
